In [3]:
import os
import pandas as pd
import requests
import numpy as np
import openai

API_KEY = os.getenv("AIMLAPI_API_KEY2")
print(API_KEY)

9eb6cce1b44e443cb7245aded35737be


In [4]:
DATASET_NAME = "embedding_index_3m.json"

def load_dataset(source: str) -> pd.core.frame.DataFrame:
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

pd_vectors = load_dataset(DATASET_NAME)
print("\nНазви колонок:")
print(pd_vectors.columns)


Назви колонок:
Index(['speaker', 'title', 'videoId', 'start', 'seconds', 'summary', 'ada_v2'], dtype='object')


In [7]:
model = 'text-embedding-ada-002'
SIMILARITIES_RESULTS_THRESHOLD = 0.75


def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(query: str, dataset: pd.core.frame.DataFrame, rows: int) -> pd.core.frame.DataFrame:
    video_vectors = dataset.copy()
    
    response = requests.post(
        "https://api.aimlapi.com/v1/embeddings",
        headers={
            "Authorization": f"Bearer {API_KEY}",
            "Content-Type": "application/json"
        },
        json={
            "model": model,
            "input": query,
            "encoding_format": "float"
        }
    )
    
    data = response.json()
    query_embeddings = data["data"][0]["embedding"]
    
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )
    
    filtered_videos = video_vectors[video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD]
    
    return filtered_videos.sort_values("similarity", ascending=False).head(rows)

def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nВідео, схожі на '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Резюме: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Схожість: {row['similarity']:.3f}")
        print(f"   Доповідачі: {row['speaker']}")




while True:
    query = input("\nВведіть запит (або 'exit' для виходу): ")
    if query.lower() == 'exit':
        break
    
    videos = get_videos(query, pd_vectors, 5)
    
    if len(videos) > 0:
        display_results(videos, query)
    else:
        print(f"Не знайдено результатів для запиту '{query}'")


Відео, схожі на 'Що таке нейронна мережа?':
 - Neuroscience with Azure Machine Learning
   Резюме: Sonya Copyof interviews Nyle Hein, a bioengineering PhD student, about using Azure Machine Learning in...
   YouTube: https://youtu.be/EeooaRAhX-U?t=0
   Схожість: 0.764
   Доповідачі: Sonya copyof, nyle hein
 - The Simplest Machine Learning
   Резюме: Neural networks are a type of design used in deep learning, which implements algorithms. Activation...
   YouTube: https://youtu.be/kUV045QBWJg?t=183
   Схожість: 0.752
   Доповідачі: Micheline Harris

Відео, схожі на 'Чи можу я використовувати Jupyter Notebooks з Azure Machine Learning?':
 - Experimentation Using Notebooks in Azure ML with Remote Compute [Part 3/4]
   Резюме: In this episode of the AI Show, the focus is on using Notebooks in Azure...
   YouTube: https://youtu.be/jey9EWKSBwM?t=0
   Схожість: 0.808
   Доповідачі: 
 - Edit and run Jupyter notebooks without leaving Azure Machine Learning studio
   Резюме: Abe Omorogbe, a Prog

In [6]:
query = "Reinforcement Learning"
top_n = 50

rl_videos = get_videos(query, pd_vectors, top_n)

if len(rl_videos) > 0:
    print(f"Знайдено {len(rl_videos)} фрагментів, пов'язаних з '{query}'\n")
    unique_speakers = rl_videos['speaker'].unique()
    
    print("Спікери, які згадували 'Reinforcement Learning':")
    for i, speaker in enumerate(unique_speakers, 1):
        print(f"{i}. {speaker}")
    
    speaker_counts = rl_videos['speaker'].value_counts()
    print("\nКількість згадок по спікерам:")
    print(speaker_counts)
    
    print("\nТоп-5 найрелевантніших результатів:")
    display_results(rl_videos.head(5), query)
else:
    print(f"Не знайдено результатів для запиту '{query}'")

Знайдено 50 фрагментів, пов'язаних з 'Reinforcement Learning'

Спікери, які згадували 'Reinforcement Learning':
1. Dr. Katja Hofmann, Seth
2. Kenji Kanazawa
3. Eric Boyd
4. Jason Williams
5. Seth Juarez
6. Moritz Steller, Setu Chokshi
7. Seth, Seth Juarez
8. Mehrnoosh Sameki
9. Christian Vance burger, Krishnan
10. Rafal Lukawiecki
11. Micheleen Harris
12. Jeremy Howard
13. Seth welcomes Microsoft Data Scientist, Spyros Marketos, ML Engineer, Davide Fornelli and Data Engineer, Samarendra Panda
14. Alex Bocharov, Principal Researcher at Microsoft Quantum Systems group and Chris Granade, Senior Research Software Development Engineer
15. 
16. Sarah Bird
17. Andreas Müller, Seth Juarez
18. Joy Qiao, Mario Inchiosa, Mario Schlener, Jason Wong, William Chan, Alex Miles
19. Dennis Eikelenboom
20. Gianpaolo, Alessandro, Gianluca
21. Paige, Lo
22. Satya Mallick, Ye Lu, Him Wai Ng, Seth
23. Micheline Harris
24. Dmytro Dzhulgakov, Seth
25. Nilesh Acharya
26. Anna Thomas
27. Timm Walz, Seth Juarez
